# 2.1 Testing The Model 

The result fo this file is 
![demo](./Assets/model-demo.gif)

### The code for acheiving above

In [1]:
# Standard Imports
import numpy as np
import cv2

In [2]:
from keras.models import load_model

Defining the constants

In [3]:
IMAGE_SIZE = 32  # Image size 32 x 32 px
THRESHOLD = 0.90  # only if the model is sure more than 90% then only give the prediction
GREEN = (0, 255, 0)  # Green Color
RED = (50, 50, 255)  # Red Color
FONT = cv2.FONT_HERSHEY_COMPLEX
WITH_MASK = 0
WITHOUT_MASK = 1

Loading the Model

In [4]:
model = load_model('./Data/mask_detection_model.model')

Instanciating the frontal face detector, using [haarcascade_frontalface_default.xml](https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml) by openCV

In [5]:
face_detector = cv2.CascadeClassifier('./Data/haarcascade_frontalface_default.xml')

## Defining the necessary Functions

In [6]:
def image_preprocessing(image):
    image = image.astype("uint8")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.equalizeHist(image)
    image = image / 255
    return image

In [7]:
def highlight_face(image, color, text, x, y, w, h):
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 5)
    cv2.rectangle(image, (x, y-40), (x+w, y), color, -2)
    cv2.putText(
        image,
        text,
        (x, y-10),
        FONT, 0.75,
        (255, 255, 255),
        1, cv2.LINE_AA
    )

## Taking the feed from webcam and checking if the faces in that feed are wearing a mask or not

In [9]:
webcam = cv2.VideoCapture(0)

# run until 'q' or 'Q' is pressed
while True:
    sucess, frame = webcam.read()  # Read the current frame
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converting the image to grey for better face recognition
    faces = face_detector.detectMultiScale(frame_gray, 1.3, 5)  # Detect for faces in the current frame
    
    for x, y, w, h in faces: # for x_cor, y_cor, width, height of every face
        crop_img = frame[y: y+h, x: x+h]  # crop the frame just to the face using list slicing
        img = cv2.resize(crop_img, (IMAGE_SIZE, IMAGE_SIZE)) # Resize the Image to the dimensions
        img = image_preprocessing(img)  
        img = img.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 1)  
        
        # Pridict for the face (if it is wearing mask or not)
        prediction = model.predict(img)
        class_ = np.argmax(prediction)  # getting the index of the hightes prediction
        probability = np.max(prediction)
        
        if probability >= THRESHOLD:
            if class_ == WITH_MASK:  # If the face has mask display 'Mask' with green rectangle
                highlight_face(frame, GREEN, 'Mask', x, y, w, h)
            elif class_ == WITHOUT_MASK:  # If the face does not have mask display 'No Mask' with red rectangle
                highlight_face(frame, RED, 'No Mask', x, y, w, h)
                
    cv2.imshow("Webcam", frame)
    key = cv2.waitKey(1) # wait for 1 milisecond and capture the key if any pressedf
    if key == ord('q') or key == ord('Q'):
        break
        
webcam.release()
cv2.destroyAllWindows()

# Done!, now lets move to the next step